In [67]:
import pandas as pd
import numpy as np
import seaborn as sns
import json

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.dates as mdates
from matplotlib import font_manager
from matplotlib.ticker import FixedLocator

from timple.timedelta import strftimedelta


import fastf1 as f1
import fastf1.plotting
from fastf1.core import Laps
from fastf1.ergast import Ergast
from fastf1 import utils

ergast = Ergast(result_type='pandas')

with open("../season_info.json") as file:
    season_info = json.loads(file.read())
teams = season_info["teams"]
team_colours = season_info["team_colours"]
tyre_colours = season_info["tyre_colours"]
drivers = season_info["drivers"]


fastf1.plotting.setup_mpl(mpl_timedelta_support=True, color_scheme="fastf1", misc_mpl_mods=True)

font_files = font_manager.findSystemFonts(fontpaths=None, fontext="ttf")

for font_file in font_files:
    font_manager.fontManager.addfont(font_file)

font = font_manager.FontProperties(family="Formula1", weight="normal", style="normal", size=16)

In [19]:
data = f1.get_session(year=2024, gp="Bahrain", identifier="R")
data.load()


core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.3.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']


In [20]:
laps = data.laps

In [21]:
all_laps_by_team = dict()

for team1 in teams:

    team_data = laps[laps["Team"] == team1]

    all_laps_by_team[team1] = team_data.reset_index()

In [1]:
'''LAP TIME PLOTTING'''


'''setup'''
def lap_time_vs_number(drivers_in_graph: list):
    axis_fontsize = 18
    tick_fontsize = 14


    '''axis settings'''
    plt.figure(figsize=(16,8), dpi=120, facecolor="#0F0F0F")
    plt.yticks(color="white", fontsize=tick_fontsize, fontname="Formula1")
    plt.xticks(color="white", fontsize=tick_fontsize, fontname="Formula1")
    plt.xlabel("Lap Number", fontsize=axis_fontsize, color="white", fontname="Formula1")
    plt.ylabel("Lap Time", fontsize=axis_fontsize, color="white", fontname="Formula1")

    ax = plt.gca()
    ax.patch.set_facecolor("#000000")

    ax.set_axisbelow(True)
    ax.yaxis.grid(True, which="major", linestyle="--", color="white", zorder=-1000)
    plt.ylim(bottom=0.001065 , top=0.00116)
    # plt.xlim(0, 332)

    '''tyre data for any drivers in the list'''

    teams_in_graph = list()
    handles = list()

    for driver in drivers_in_graph:
        driver = driver.upper()
        current_team = drivers[driver]["Team"]
        team_colour = team_colours[current_team]
        team_data = all_laps_by_team[current_team]
        driver_laps = team_data[team_data["Driver"] == driver].reset_index()

        # if two drivers are of the same team, the dots in the scatter are squares
        # also gives them the legend handle with the correct colour and name
        if current_team not in teams_in_graph:
            marker = "o"
            dot_size = 65
            handle = mlines.Line2D([],[], linestyle="None", color=team_colour, marker=marker, markersize=12, label=driver)
        else:
            marker = "*"
            dot_size = 100
            handle = mlines.Line2D([],[], linestyle="None", color=team_colour, marker=marker, markersize=12, label=driver)

        teams_in_graph.append(current_team)
        handles.append(handle)


        # splitting drivers laps by tyre
        laps_by_tyre = {
            "soft": driver_laps[driver_laps["Compound"] == "SOFT"],
            "medium": driver_laps[driver_laps["Compound"] == "MEDIUM"],
            "hard": driver_laps[driver_laps["Compound"] == "HARD"]
        }
        
        for tyre, tyre_laps in laps_by_tyre.items():
            ax.scatter(
                tyre_laps.LapNumber, tyre_laps.LapTime,
                color=team_colour, s=dot_size, linewidths=1, edgecolors=tyre_colours[tyre], marker=marker,
            )

    handle = mlines.Line2D(
        [], [], linestyle="None",
        marker="o", markerfacecolor="None", markeredgecolor=tyre_colours["medium"], markersize=12,
        label="Tyre"
    )
    handles.append(handle)

    ax.legend(handles=handles, prop=font)

    title = "Race Pace Comparison\n"
    for driver in drivers_in_graph:
        title = title + f"{drivers[driver]["Full Name"]}, "
    title = title[:-2]

    plt.suptitle(title, fontsize=16, fontname="Formula1", color="white")
    # ax.legend(drivers_in_graph)

    plt.show()


In [2]:
drivers_in_graph = ["LEC", "VER"]

lap_time_vs_number(drivers_in_graph)

NameError: name 'plt' is not defined

In [60]:
all_laps_by_team["McLaren"]

,index,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,...,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
0,285,0 days 01:01:42.394000,NOR,4,0 days 00:01:42.168000,1.0,1.0,NaT,NaT,NaT,...,False,McLaren,0 days 00:59:59.911000,2024-03-02 15:03:42.342,2,7.0,False,,False,False
1,286,0 days 01:03:20.404000,NOR,4,0 days 00:01:38.010000,2.0,1.0,NaT,NaT,0 days 00:00:31.353000,...,False,McLaren,0 days 01:01:42.394000,2024-03-02 15:05:24.825,1,7.0,False,,False,True
2,287,0 days 01:04:58.191000,NOR,4,0 days 00:01:37.787000,3.0,1.0,NaT,NaT,0 days 00:00:31.112000,...,False,McLaren,0 days 01:03:20.404000,2024-03-02 15:07:02.835,1,6.0,True,TRACK LIMITS AT TURN 15 LAP 3,False,True
3,288,0 days 01:06:35.641000,NOR,4,0 days 00:01:37.450000,4.0,1.0,NaT,NaT,0 days 00:00:31.115000,...,False,McLaren,0 days 01:04:58.191000,2024-03-02 15:08:40.622,1,6.0,True,TRACK LIMITS AT TURN 15 (NEXT LAP),False,True
4,289,0 days 01:08:13.435000,NOR,4,0 days 00:01:37.794000,5.0,1.0,NaT,NaT,0 days 00:00:31.258000,...,False,McLaren,0 days 01:06:35.641000,2024-03-02 15:10:18.072,1,6.0,False,,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,451,0 days 02:26:16.950000,PIA,81,0 days 00:01:35.739000,53.0,3.0,NaT,NaT,0 days 00:00:30.770000,...,True,McLaren,0 days 02:24:41.211000,2024-03-02 16:28:23.642,1,8.0,False,,False,True
110,452,0 days 02:27:53.057000,PIA,81,0 days 00:01:36.107000,54.0,3.0,NaT,NaT,0 days 00:00:30.976000,...,True,McLaren,0 days 02:26:16.950000,2024-03-02 16:29:59.381,1,8.0,False,,False,True
111,453,0 days 02:29:29.030000,PIA,81,0 days 00:01:35.973000,55.0,3.0,NaT,NaT,0 days 00:00:30.879000,...,True,McLaren,0 days 02:27:53.057000,2024-03-02 16:31:35.488,1,8.0,False,,False,True
112,454,0 days 02:31:05.130000,PIA,81,0 days 00:01:36.100000,56.0,3.0,NaT,NaT,0 days 00:00:30.820000,...,True,McLaren,0 days 02:29:29.030000,2024-03-02 16:33:11.461,1,8.0,False,,False,True


In [68]:
pit_stops = ergast.get_pit_stops(2024, 1)

In [70]:
pit_stops.content

[           driverId  stop  lap      time               duration
 0        hulkenberg     1    1  18:05:33 0 days 00:00:36.604000
 1              zhou     1    9  18:18:39 0 days 00:00:25.208000
 2            stroll     1    9  18:18:42 0 days 00:00:24.418000
 3              ocon     1   10  18:20:20 0 days 00:00:24.877000
 4          sargeant     1   10  18:21:19 0 days 00:00:35.729000
 5           russell     1   11  18:21:38 0 days 00:00:24.778000
 6           leclerc     1   11  18:21:41 0 days 00:00:24.069000
 7   kevin_magnussen     1   11  18:21:55 0 days 00:00:25.951000
 8             perez     1   12  18:23:16 0 days 00:00:24.516000
 9           piastri     1   12  18:23:24 0 days 00:00:25.450000
 10         hamilton     1   12  18:23:26 0 days 00:00:25.087000
 11            gasly     1   12  18:23:38 0 days 00:00:29.980000
 12           bottas     1   12  18:23:42 0 days 00:00:26.420000
 13           norris     1   13  18:25:00 0 days 00:00:24.205000
 14        ricciardo     